# SM2RAIN - Global Precipitation Data

In [1]:
import rioxarray
import xarray
from pathlib import Path
import datetime
from eumap.parallel import job
from eumap.misc import find_files, ttprint
from eumap.raster import read_rasters, save_rasters
from eumap.misc import nan_percentile
import gdal
import os

In [2]:
SM2RAIN_DIR = '/mnt/tupi/SM2RAIN/v1.5/'
VARIABLE = 'Rainfall'
OUPUT_DIR = '/mnt/tupi/SM2RAIN/'
START_YEAR = 2007
END_YEAR = 2021

In [78]:
def set_scale(raster_file, scale):
    os.system(f'gdal_edit.py -scale {1/scale} {raster_file}')

## GeoTIFF conversion

In [12]:
import calendar

args = []
for nc_file in find_files(SM2RAIN_DIR, '*.nc'):
    year = nc_file.name.split('_')[3]
    n_days = 365
    if calendar.isleap(int(year)):
        n_days += 1
    args += [ (nc_file, VARIABLE, year, doy, Path(OUPUT_DIR)) for doy in range(0,n_days) ]

In [18]:
#import os
#os.environ['HDF5_USE_FILE_LOCKING'] = 'FALSE'

def nc2geoatiff(nc_file, nc_var, year, doy, out_dir, dtpye='uint16', nodata=65365, scale = 10):
        
    dt = datetime.datetime.strptime(f'{year} {str(doy+1).zfill(3)}', '%Y %j')
    dts = dt.strftime('%Y.%m.%d')
    dty = dt.strftime('%Y')
    
    geotiff_file = out_dir.joinpath(f'clm_precipitation_sm2rain.daily_{dty}_v1.5').joinpath(f'clm_precipitation_sm2rain.daily_m_10km_s0..0cm_{dts}_v1.5.tif')
    geotiff_file.parent.mkdir(parents=True, exist_ok=True)
    
    xds = xarray.open_dataset(nc_file)
    xds.rio.write_crs("epsg:4326", inplace=True)
    
    data = (xds[nc_var][:,:,doy] * scale).fillna(nodata)
    data.rio.set_attrs(xds.attrs, inplace=True)
    data.rio.write_nodata(nodata, inplace=True)
    
    data.rio.to_raster(geotiff_file, dtype='uint16', nodata=nodata, tiled=True, compress='deflate')
    
    #Produces inconsistent values for the nodata in the next processing steps
    set_scale(geotiff_file, scale)
    
    return geotiff_file

ttprint(f"Start processing")
output_files = []
for geotiff_file in job(nc2geotiff, args):
    output_files.append(output_files)
    #if len(output_files) % 100 == 0:
    #    ttprint(f"Processing progress: {len(output_files)}/{len(args)}")
ttprint(f"End of processing")

[17:06:20] Start processing


/home/opengeohub/.local/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[17:10:12] End of processing


## Monthly Aggregations

In [11]:
args = []
for year in range(START_YEAR,END_YEAR+1):
    for month in range(1,13):
        raster_files = find_files(Path(OUPUT_DIR).joinpath(f'clm_precipitation_sm2rain.daily_{year}_v1.5'), 
                                  f'clm_precipitation_sm2rain.daily_m_10km_s0..0cm_{year}.{str(month).zfill(2)}*_v1.5.tif')
        args += [ (year, month, raster_files, Path(OUPUT_DIR))]

In [11]:
#raster_files = ['/mnt/tupi/SM2RAIN/clm_precipitation_sm2rain.daily_2020_v1.5/clm_precipitation_sm2rain.daily_m_10km_s0..0cm_2020-12-30_v1.5.tif']
#data, _ = read_rasters(raster_files=raster_files, dtype='float32')
#save_rasters(raster_files[0], ['/mnt/tupi/SM2RAIN/teste.tif'], data, dtype='float32')

['/mnt/tupi/SM2RAIN/teste.tif']

In [13]:
import gdal
import numpy as np
import calendar
import bottleneck as bc

def monthlyAgg(year, month, raster_files, out_dir, q_arr=[5, 50, 95], nodata=65365, scale=10):
    data, _ = read_rasters(raster_files = raster_files, dtype='float32')
    
    nan_mask = np.all(np.isnan(data), axis=-1)
    data_sum = np.stack([ bc.nansum(data, axis=-1) ], axis=-1)
    data_sum[nan_mask] = np.nan
    
    data_avg = np.stack([ bc.nanmean(data, axis=-1) ], axis=-1)
    data_std = np.stack([ bc.nanstd(data, axis=-1) ], axis=-1)
    
    data_perc = nan_percentile(data.transpose(2,0,1), q=q_arr)
    data_perc = data_perc.transpose(1,2,0).astype('float32')
    
    data_perc = np.concatenate([data_perc, data_avg, data_std, data_sum], axis=-1)
    
    dt_start = Path(raster_files[0]).name.split('_')[6]
    dt_end = Path(raster_files[-1]).name.split('_')[6]
    
    out_files = [ 
        out_dir.joinpath(f'clm_precipitation_sm2rain.monthly_p{str(q).zfill(2)}_{START_YEAR}..{END_YEAR}_v1.5') \
            .joinpath(f'clm_precipitation_sm2rain.monthly_p{str(q).zfill(2)}_10km_s0..0cm_{dt_start}..{dt_end}_v1.5.tif') 
        for q in q_arr 
    ]
    out_files += [ 
        out_dir.joinpath(f'clm_precipitation_sm2rain.monthly_m_{START_YEAR}..{END_YEAR}_v1.5') \
            .joinpath(f'clm_precipitation_sm2rain.monthly_m_10km_s0..0cm_{dt_start}..{dt_end}_v1.5.tif') ,
        
        out_dir.joinpath(f'clm_precipitation_sm2rain.monthly_sd_{START_YEAR}..{END_YEAR}_v1.5') \
            .joinpath(f'clm_precipitation_sm2rain.monthly_sd_10km_s0..0cm_{dt_start}..{dt_end}_v1.5.tif') ,
        
        out_dir.joinpath(f'clm_precipitation_sm2rain.monthly_sum_{START_YEAR}..{END_YEAR}_v1.5') \
            .joinpath(f'clm_precipitation_sm2rain.monthly_sum_10km_s0..0cm_{dt_start}..{dt_end}_v1.5.tif'),
    ]
    
    save_rasters(raster_files[0], out_files, data_perc)
    for out_file in out_files:
        set_scale(out_file, scale)
    
    return out_files

output_files = []

ttprint(f"Start processing")
for output_file in job(monthlyAgg, args):
    output_files.append(output_file)
    #if len(output_files) % 10 == 0:
    #    ttprint(f"Processing progress: {len(output_files)}/{len(args)}")
ttprint(f"End of processing")

[10:04:38] Start processing
[10:11:11] End of processing


## Yearly Aggregations

In [32]:
args = []
for month in range(1,13):
    raster_files = find_files(Path(OUPUT_DIR).joinpath(f'clm_precipitation_sm2rain.monthly_sum_{START_YEAR}..{END_YEAR}_v1.5'),
                              f'clm_precipitation_sm2rain.monthly_sum_10km_s0..0cm_????.{str(month).zfill(2)}*_v1.5.tif')
    month = datetime.datetime.strptime(f'{START_YEAR}.{month}.1', '%Y.%m.%d').strftime('%b').lower()
    args += [ (month, START_YEAR, END_YEAR, raster_files, Path(OUPUT_DIR))]

In [33]:
import gdal
import numpy as np
import calendar
import bottleneck as bc

def yearlyAgg(month, start_year, end_year, raster_files, out_dir, q_arr=[5, 50, 95], nodata=65365, scale=10):
    data, _ = read_rasters(raster_files = raster_files, dtype='float32', n_jobs=20)
    
    data_avg = np.stack([ bc.nanmean(data, axis=-1) ], axis=-1)
    data_std = np.stack([ bc.nanstd(data, axis=-1) ], axis=-1)
    
    data_perc = nan_percentile(data.transpose(2,0,1), q=q_arr)
    data_perc = data_perc.transpose(1,2,0).astype('float32')
    data_perc = np.concatenate([data_perc, data_avg, data_std], axis=-1)
    
    #out_dir = out_dir
    #out_dir.parent.mkdir(parents=True, exist_ok=True)
    
    out_files = [ 
        out_dir.joinpath(f'clm_precipitation_sm2rain.sum_p{str(q).zfill(2)}_{START_YEAR}..{END_YEAR}_v1.5') \
            .joinpath(f'clm_precipitation_sm2rain.sum.{month}_p{str(q).zfill(2)}_10km_s0..0cm_{start_year}..{end_year}_v1.5.tif') for q in q_arr 
    ]
    
    out_files += [
        out_dir.joinpath(f'clm_precipitation_sm2rain.sum_m_{START_YEAR}..{END_YEAR}_v1.5') \
            .joinpath(f'clm_precipitation_sm2rain.sum.{month}_m_10km_s0..0cm_{start_year}..{end_year}_v1.5.tif'),
        out_dir.joinpath(f'clm_precipitation_sm2rain.sum_std_{START_YEAR}..{END_YEAR}_v1.5') \
            .joinpath(f'clm_precipitation_sm2rain.sum.{month}_sd_10km_s0..0cm_{start_year}..{end_year}_v1.5.tif') 
    ]
    
    save_rasters(raster_files[0], out_files, data_perc)
    for out_file in out_files:
        set_scale(out_file, scale)
    
    return out_files

output_files = []

ttprint(f"Start processing")
for output_file in job(yearlyAgg, args):
    output_files.append(output_file)
ttprint(f"End of processing")

[11:27:12] Start processing
[11:27:33] End of processing


## Trend analysis

In [4]:
raster_files = find_files(Path(OUPUT_DIR).joinpath(f'clm_precipitation_sm2rain.monthly_sum_{START_YEAR}..{END_YEAR}_v1.5'), f'*.tif')
data, _ = read_rasters(raster_files=raster_files, n_jobs=40, dtype='float32')
data.shape

In [5]:
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.seasonal import STL
from scipy.special import logit
from eumap import parallel

def apply_along_axis(
  worker,
  axis,
  arr,
  *args:any,
  **kwargs:any
):
 
  import numpy as np

  def run(worker, axis, arr, args, kwargs):
    return np.apply_along_axis(worker, axis, arr, *args, **kwargs)

 
  effective_axis = 1 if axis == 0 else axis
  if effective_axis != axis:
      arr = arr.swapaxes(axis, effective_axis)

  # Chunks for the mapping (only a few chunks):
  chunks = [(worker, effective_axis, sub_arr, args, kwargs)
            for sub_arr in np.array_split(arr, 100)]

  result = []
  for r in job(run, chunks):
    result.append(r)

  return np.concatenate(result)

def trend_analysis(data):
    from scipy.special import expit, logit
    from statsmodels.tsa.seasonal import STL

    has_nan = np.sum(np.isnan(data).astype('int'))
    
    if has_nan == 0:
        
        res = STL(data, period=12, seasonal=13, trend=25, robust=True).fit()
        
        trend = res.trend

        trend[trend > 30000] = 30000
        trend[trend < 0] = 0
        
        trend_norm = (trend + 1) / 30002
        trend_norm = logit(trend_norm)
        
        y = trend_norm
        y_size = trend_norm.shape[0]
        
        X = np.array(range(0, y_size)) / y_size

        X = sm.add_constant(X)
        model = sm.OLS(y,X)
        results = model.fit()
        
        conf_int = results.conf_int(alpha=0.05, cols=None)
        
        result_stack = np.stack([
            results.params,
            results.bse,
            results.tvalues,
            results.pvalues,
            conf_int[0],
            conf_int[1]
        ],axis=1)
        
        return np.concatenate([
            trend,
            result_stack[0,:],
            result_stack[1,:],
            np.stack([results.rsquared])
        ])
        
    else: 
        nan_result = np.empty(193)
        nan_result[:] = np.nan
        return nan_result

ttprint(f"Start processing")
trend_result = apply_along_axis(trend_analysis, 2, data)
ttprint(f"End processing")

[12:01:14] Start processing


/opt/conda/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/opt/conda/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/opt/conda/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/opt/conda/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/opt/conda/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/opt/conda/lib/python3.8/site-packages/statsmodels/regression/linear_model.

[12:17:29] End processing


In [54]:
out_dir = Path(OUPUT_DIR).joinpath('clm_precipitation_sm2rain.trend_sum_2007..2021_v1.5')
out_files = [ out_dir.joinpath(str(r.name).replace('sm2rain.monthly', 'sm2rain.trend')) for r in raster_files ]

out_dir = Path(OUPUT_DIR).joinpath('clm_precipitation_sm2rain.trend.logit.ols_2007..2021_v1.5')
out_files += [
     out_dir.joinpath('clm_precipitation_sm2rain.trend.logit.ols.alpha_m_10km_s0..0cm_2007..2007.01.31_v1.5.tif'),
     out_dir.joinpath('clm_precipitation_sm2rain.trend.logit.ols.alpha_sd_10km_s0..0cm_2007..2007.01.31_v1.5.tif'),
     out_dir.joinpath('clm_precipitation_sm2rain.trend.logit.ols.alpha_tv_10km_s0..0cm_2007..2007.01.31_v1.5.tif'),
     out_dir.joinpath('clm_precipitation_sm2rain.trend.logit.ols.alpha_pv_10km_s0..0cm_2007..2007.01.31_v1.5.tif'),
     out_dir.joinpath('clm_precipitation_sm2rain.trend.logit.ols.alpha_l.025_10km_s0..0cm_2007..2007.01.31_v1.5.tif'),
     out_dir.joinpath('clm_precipitation_sm2rain.trend.logit.ols.beta_l.025_10km_s0..0cm_2007..2007.01.31_v1.5.tif'),
     out_dir.joinpath('clm_precipitation_sm2rain.trend.logit.ols.beta_m_10km_s0..0cm_2007..2007.01.31_v1.5.tif'),
     out_dir.joinpath('clm_precipitation_sm2rain.trend.logit.ols.beta_sd_10km_s0..0cm_2007..2007.01.31_v1.5.tif'),
     out_dir.joinpath('clm_precipitation_sm2rain.trend.logit.ols.beta_tv_10km_s0..0cm_2007..2007.01.31_v1.5.tif'),
     out_dir.joinpath('clm_precipitation_sm2rain.trend.logit.ols.beta_pv_10km_s0..0cm_2007..2007.01.31_v1.5.tif'),
     out_dir.joinpath('clm_precipitation_sm2rain.trend.logit.ols.alpha_u.975_10km_s0..0cm_2007..2007.01.31_v1.5.tif'),
     out_dir.joinpath('clm_precipitation_sm2rain.trend.logit.ols.beta_u.975_10km_s0..0cm_2007..2007.01.31_v1.5.tif'),
     out_dir.joinpath('clm_precipitation_sm2rain.trend.logit.ols_r2_10km_s0..0cm_2007..2007.01.31_v1.5.tif')
]
len(out_files)

193

In [58]:
i = 180
save_rasters(raster_files[0], out_files[0:i], trend_result[:,:,0:i], n_jobs=10)
for out_file in out_files[0:i]:
    set_scale(out_file, 10)

In [79]:
save_rasters(raster_files[0], out_files[i:], (trend_result[:,:,i:] * 100), n_jobs=10, dtype='int16', nodata=32767)
for out_file in out_files[i:]:
    set_scale(out_file, 100)